In [178]:
import pandas as pd
import numpy as np

In [179]:
df = pd.read_csv("./data/Turnstiles/Turnstile_Usage_Data__2020.csv")
df.columns = df.columns.str.rstrip() # remove spaces at the end of Exits column name

In [180]:
df = df.sort_values(["Station", "Line Name", "Date", "Time"])

In [181]:
df.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
31897,H007,R248,00-00-00,1 AV,L,BMT,01/01/2020,03:00:00,REGULAR,15313601,17119864
31939,H007,R248,00-00-01,1 AV,L,BMT,01/01/2020,03:00:00,REGULAR,60962942,38072592
31981,H007,R248,00-03-00,1 AV,L,BMT,01/01/2020,03:00:00,REGULAR,370821688,387998673
32023,H007,R248,00-03-01,1 AV,L,BMT,01/01/2020,03:00:00,REGULAR,2572773,1111614
32065,H007,R248,00-03-02,1 AV,L,BMT,01/01/2020,03:00:00,REGULAR,6582077,554497


# Step 1 Get Max daily value for each device

Group by `Station` > `Line Name` > `Date` > `SCP` 

note: `SCP` = (specific address for a device (02-00-00))

Example issue we need to solve: take sample data below. Happens when the turnstile device resets:

Station | Line | Date |	Time |	Entries |
---|---|---|---|---|
Station A |	ABC |	01/01/2019 |	1:00 |	1000 |
Station A |	ABC |	01/01/2019 |	3:00 |	1100 |		
Station A |	ABC |	01/01/2019 |	6:00 |	**1200** |		
Station A |	ABC |	01/01/2019 |	9:00 |	**100**	 |	
Station A |	ABC |	01/01/2019 |	12:00 |	200	 |


real situation: `df[(df["Station"] == "ATL AV-BARCLAY") & (df["SCP"] == '00-04-02') &( df["Date"] =='02/18/2020')]`
```
	C/A	Unit	SCP	Station	Line Name	Division	Date	Time	Description	Entries	Exits
1636833	R610	R057	00-04-02	ATL AV-BARCLAY	2345BDNQR	IRT	02/18/2020	03:00:00	REGULAR	114604	76009
1636834	R610	R057	00-04-02	ATL AV-BARCLAY	2345BDNQR	IRT	02/18/2020	07:00:00	REGULAR	114660	76015
1636835	R610	R057	00-04-02	ATL AV-BARCLAY	2345BDNQR	IRT	02/18/2020	11:00:00	REGULAR	114828	76046
1636836	R610	R057	00-04-02	ATL AV-BARCLAY	2345BDNQR	IRT	02/18/2020	15:00:00	REGULAR	16050	5445
1636837	R610	R057	00-04-02	ATL AV-BARCLAY	2345BDNQR	IRT	02/18/2020	19:00:00	REGULAR	16075	5495
1636838	R610	R057	00-04-02	ATL AV-BARCLAY	2345BDNQR	IRT	02/18/2020	23:00:00	REGULAR	16102	5538
```

Want the logic to be: 
```js
if last()> first()
? last() - first() 
: 0 // used to be : max()-first() + last() but then seems that they don't reset to 0
```

----- temp experiments

In [195]:
list(df[df["Station"].str.contains("METRO")].groupby(["Station", "Line Name","SCP", "Date"]))

[(('JAY ST-METROTEC', 'ACF', '00-00-00', '01/01/2020'),
          C/A  Unit       SCP          Station Line Name Division        Date  \
  62982  N103  R127  00-00-00  JAY ST-METROTEC       ACF      IND  01/01/2020   
  62983  N103  R127  00-00-00  JAY ST-METROTEC       ACF      IND  01/01/2020   
  62984  N103  R127  00-00-00  JAY ST-METROTEC       ACF      IND  01/01/2020   
  62985  N103  R127  00-00-00  JAY ST-METROTEC       ACF      IND  01/01/2020   
  62986  N103  R127  00-00-00  JAY ST-METROTEC       ACF      IND  01/01/2020   
  62987  N103  R127  00-00-00  JAY ST-METROTEC       ACF      IND  01/01/2020   
  
             Time Description  Entries    Exits  
  62982  03:00:00     REGULAR  1774972  6628661  
  62983  07:00:00     REGULAR  1774983  6628710  
  62984  11:00:00     REGULAR  1775009  6628783  
  62985  15:00:00     REGULAR  1775056  6628939  
  62986  19:00:00     REGULAR  1775104  6629106  
  62987  23:00:00     REGULAR  1775131  6629210  ),
 (('JAY ST-METROTEC', 

In [202]:
df[df["Station"].str.contains("7 AV")].groupby(["Line Name",  "Unit"]).size()

Line Name  Unit
BDE        R113     7951
BQ         R171     3196
FG         R288     5634
MR         R219     7984
NQRW       R080    15939
dtype: int64

---- end temp experiments

In [112]:
scpDailyValues = df.groupby(["Station", 
                                 "Line Name", 
                                 "SCP", 
                                 "Date"
                                ])["Time", "Entries", "Exits"].agg(["last", "first", "max"])

In [113]:
scpDailyValues['Daily Entries'] = np.where(scpDailyValues['Entries']['last'] > scpDailyValues['Entries']['first']
                                           , scpDailyValues['Entries']['last'] - scpDailyValues['Entries']['first']
                                           , 0)
scpDailyValues['Daily Exits'] = np.where(scpDailyValues['Exits']['last'] > scpDailyValues['Exits']['first']
                                           , scpDailyValues['Exits']['last'] - scpDailyValues['Exits']['first']
                                           , 0)

In [114]:
scpDailyValues.head()

Time                       Entries  \
                                           last     first       max      last   
Station Line Name SCP      Date                                                 
1 AV    L         00-00-00 01/01/2020  23:00:00  03:00:00  23:00:00  15314610   
                           01/02/2020  23:00:00  03:00:00  23:00:00  15317849   
                           01/03/2020  23:00:00  03:00:00  23:00:00  15321202   
                           01/04/2020  23:00:00  03:00:00  23:00:00  15323355   
                           01/05/2020  23:00:00  03:00:00  23:00:00  15325160   

                                                              Exits            \
                                          first       max      last     first   
Station Line Name SCP      Date                                                 
1 AV    L         00-00-00 01/01/2020  15313601  15314610  17121748  17119864   
                           01/02/2020  15314633  15317849  17125437  17121822   
                           01/03/2020  15318008  15321202  17129500  17125655   
                           01/04/2020  15321494  15323355  17133263  17129857   
                           01/05/2020  15323654  15325160  17136349  17133621   

                                                Daily Entries Daily Exits  
                                            max                            
Station Line Name SCP      Date                                            
1 AV    L         00-00-00 01/01/2020  17121748          1009        1884  
                           01/02/2020  17125437          3216        3615  
                           01/03/2020  17129500          3194        3845  
                           01/04/2020  17133263          1861        3406  
                           01/05/2020  17136349          1506        2728

## Step 2: Get daily value for each station/line

Group by `Station` > `Line Name` > `Date` — aggreagte by summing value

In [174]:
stationDailyValues = (scpDailyValues[["Daily Entries", "Daily Exits"]]
                      .reset_index(level=0)
                      .groupby(["Station", "Line Name", "Date"])
                      .sum()) # add up all the SCP daily rates together to the station level

In [177]:
stationDailyValues.reset_index().to_csv("./data/output/dailyMTAStats.csv", index=False)

In [ ]:
# Fix # 2: ANOMALYIES (all daily values turn up negative)
# Date	Description	Division	GTFS Stop ID	Line	Line Name	Ntaname	SCP	Station
# 1/2/2020	REGULAR	IND	R17	Broadway - Brighton	BDFMNQRW	Midtown-Midtown South	02-06-01	34 ST-HERALD SQ


In [ ]:
# Fix # 3: SMOOTHING

In [29]:
# grouped.to_csv("./data/output/dailyMTAStats.csv", index=False)